In [1]:
import pandas as pd
import numpy as np
import datatable as dt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import optuna

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Ignore Warings
import warnings
warnings.filterwarnings("ignore")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-oct-2021/train.csv
/kaggle/input/tabular-playground-series-oct-2021/test.csv


<font size=5>**Information about data**</font>

In [2]:
train = dt.fread('../input/tabular-playground-series-oct-2021/train.csv').to_pandas().drop('id', axis=1)
train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
0,0.205979,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,0.147295,...,False,True,False,False,False,False,False,False,False,True
1,0.181004,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,0.238509,...,False,True,False,False,False,False,False,False,False,True
2,0.182583,0.307431,0.325950,0.207116,0.605699,0.309695,0.493337,0.751107,0.536272,0.286813,...,False,False,False,True,True,False,False,False,False,True
3,0.180240,0.494592,0.008367,0.223580,0.760618,0.439211,0.432055,0.776147,0.483958,0.260886,...,False,False,False,False,True,False,False,False,False,True
4,0.177172,0.495513,0.014263,0.548819,0.625396,0.562493,0.117158,0.561255,0.077115,0.158321,...,False,True,True,False,True,False,False,True,False,True


In [3]:
test = dt.fread('../input/tabular-playground-series-oct-2021/test.csv').to_pandas().drop('id', axis=1)
ss = dt.fread('../input/tabular-playground-series-oct-2021/sample_submission.csv').to_pandas()
test.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284
0,0.178216,0.435617,0.010230,0.202074,0.390170,0.324221,0.221722,0.738894,0.582588,0.343770,...,True,False,False,False,False,False,True,True,True,False
1,0.181250,0.476455,0.022413,0.283146,0.598020,0.349508,0.283467,0.721575,0.268990,0.208373,...,False,False,False,False,False,False,False,False,False,False
2,0.159721,0.451202,0.259649,0.365274,0.594634,0.413502,0.249318,0.642339,0.411104,0.246891,...,False,False,False,False,False,False,True,False,False,False
3,0.182424,0.520976,0.095344,0.327742,0.741830,0.358711,0.270077,0.601662,0.297742,0.252829,...,False,False,False,False,False,True,True,False,False,False
4,0.229329,0.336513,0.023511,0.300913,0.668738,0.481586,0.545660,0.667849,0.546045,0.202731,...,False,False,False,False,True,False,False,True,False,False


In [4]:
r1,c1 = train.shape
print('The training data has {} rows and {} columns'.format(r1,c1))
r2,c2 = test.shape
print('The test data has {} rows and {} columns'.format(r2,c2))

The training data has 1000000 rows and 286 columns
The test data has 500000 rows and 285 columns


In [5]:
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 286 entries, f0 to target
dtypes: bool(46), float64(240)
memory usage: 1.8 GB


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,0.214334,0.460218,0.129253,0.277598,0.580710,0.416619,0.386532,0.654858,0.462256,0.258031,...,0.202336,0.207648,0.111832,0.193415,0.227804,0.020571,0.171748,0.228530,0.187746,0.220967
std,0.053320,0.101316,0.120805,0.063163,0.115338,0.058231,0.133457,0.065158,0.129439,0.119081,...,0.032468,0.038213,0.106602,0.075371,0.067996,0.073547,0.057980,0.138197,0.061176,0.070907
min,0.041790,0.022016,0.000381,0.000000,0.000000,0.000959,0.000592,0.017994,0.000990,0.051183,...,0.087593,0.000000,0.024656,0.000000,0.056023,0.000114,0.015935,0.001260,0.011777,0.007144
25%,0.181676,0.389215,0.017692,0.235342,0.497938,0.374390,0.317815,0.615372,0.363753,0.164559,...,0.194760,0.200813,0.035676,0.133782,0.172080,0.005004,0.138110,0.197519,0.142875,0.179412
50%,0.204498,0.453893,0.095496,0.264669,0.565059,0.414009,0.420260,0.648562,0.475701,0.227714,...,0.195633,0.202887,0.081907,0.167083,0.210731,0.006707,0.156161,0.202545,0.170167,0.208435
75%,0.229684,0.526023,0.177717,0.305837,0.657024,0.458360,0.477140,0.692666,0.561372,0.300988,...,0.196565,0.205090,0.140333,0.221812,0.255385,0.008481,0.188783,0.362163,0.199849,0.242721
max,1.000000,0.959019,0.994818,0.979797,1.000000,0.907889,0.972601,0.986195,0.986118,0.980994,...,0.712737,0.993422,0.996664,1.000000,1.000000,1.000000,1.000000,0.987972,0.937160,0.988138


In [6]:
# To see the quantity of null vaues in all the columns.
# c1 stands for the number of columns in the training data.

print('MISSING VALUES IN TRAINING DATASET:')
print(train.isna().sum().nlargest(c1))
print('')
print('MISSING VALUES IN TEST DATASET:')
print(test.isna().sum().nlargest(c2))

MISSING VALUES IN TRAINING DATASET:
f0        0
f188      0
f194      0
f193      0
f192      0
         ..
f101      0
f102      0
f103      0
f104      0
target    0
Length: 286, dtype: int64

MISSING VALUES IN TEST DATASET:
f0      0
f188    0
f194    0
f193    0
f192    0
       ..
f101    0
f102    0
f103    0
f104    0
f284    0
Length: 285, dtype: int64


<font size=5>**To reduce the size of data**</font>

In [7]:
'''A function to reduce the amount of memory taken up by each feature by compressing it to the appropriate datatype
verbose parameter is used to output a message regarding the exact memory usage reduction'''
def reduce_memory_usage(df, verbose=True):
    numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2 #initial memory usage to compare to
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            #extract the min and max values
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2 #new memory_usage
    if verbose:
        print(
            "Memory usage decreased to: {:.2f} Mb - {:.1f}% reduction".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
                
                )
            )
    return df

In [8]:
%%time
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)
ss = reduce_memory_usage(ss)

Memory usage decreased to: 501.63 Mb - 73.2% reduction
Memory usage decreased to: 250.34 Mb - 73.3% reduction
Memory usage decreased to: 2.86 Mb - 50.0% reduction
CPU times: user 43.4 s, sys: 1min 1s, total: 1min 44s
Wall time: 1min 45s


In [9]:
bool_cols_train = []
for i, col in enumerate(train.columns):
    if train[col].dtypes == bool:
        bool_cols_train.append(i)

In [10]:
bool_cols_test = []
for i, col in enumerate(test.columns):
    if train[col].dtypes == bool:
        bool_cols_test.append(i)

In [11]:
train.iloc[:, bool_cols_train] = train.iloc[:, bool_cols_train].astype(int)
test.iloc[:, bool_cols_test] = test.iloc[:, bool_cols_test].astype(int)

In [12]:
print("Train set shape", train.shape, "\n", "Test set shape", test.shape)

Train set shape (1000000, 286) 
 Test set shape (500000, 285)


In [13]:
train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
0,0.205933,0.410889,0.176758,0.223633,0.423584,0.476074,0.413574,0.611816,0.534668,0.147339,...,0,1,0,0,0,0,0,0,0,1
1,0.181030,0.473145,0.011734,0.213623,0.619629,0.441650,0.230347,0.686035,0.281982,0.238525,...,0,1,0,0,0,0,0,0,0,1
2,0.182617,0.307373,0.325928,0.207153,0.605469,0.309814,0.493408,0.750977,0.536133,0.286865,...,0,0,0,1,1,0,0,0,0,1
3,0.180298,0.494629,0.008369,0.223633,0.760742,0.439209,0.432129,0.776367,0.483887,0.260986,...,0,0,0,0,1,0,0,0,0,1
4,0.177124,0.495605,0.014259,0.548828,0.625488,0.562500,0.117188,0.561035,0.077087,0.158325,...,0,1,1,0,1,0,0,1,0,1


In [14]:
X = train.drop('target', axis=1).copy()
y = train['target'].copy()
X_test = test.copy()

del train
del test

In [15]:
X['std'] = X.std(axis=1)
X['min'] = X.min(axis=1)
X['max'] = X.max(axis=1)

X_test['std'] = X_test.std(axis=1)
X_test['min'] = X_test.min(axis=1)
X_test['max'] = X_test.max(axis=1)

In [16]:
params = {
    'max_depth': 6,
    'n_estimators': 9500,
    'subsample': 0.7,
    'colsample_bytree': 0.2,
    'colsample_bylevel': 0.60,
    'min_child_weight': 56.42,
    'reg_lambda': 75.57,
    'reg_alpha': 0.118,
    'gamma': 0.641,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'use_label_encoder': False
    }

In [17]:
%%time
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    params['learning_rate']=0.007
    model1 = XGBClassifier(**params)
    model1.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False)
    
    params['learning_rate']=0.03
    model2 = XGBClassifier(**params)
    model2.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False,
              xgb_model=model1)
    
    params['learning_rate']=0.15
    model3 = XGBClassifier(**params)
    model3.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False,
              xgb_model=model2)
    
    pred_valid = model3.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('._'*30)
    
    test_preds = model3.predict_proba(X_test)[:,1]
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")

Fold: 1 Score: 0.8588753600158262
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Fold: 2 Score: 0.8559630688256515
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Fold: 3 Score: 0.8578772684267219
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Fold: 4 Score: 0.8563280971691066
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Fold: 5 Score: 0.8569344633396365
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Overall Validation Score: 0.8571956515553886
CPU times: user 47min 37s, sys: 38.6 s, total: 48min 15s
Wall time: 46min 15s


In [18]:
predictions = np.mean(np.column_stack(preds),axis=1)

ss['target'] = predictions
ss.to_csv('./xgb.csv', index=False)
ss.head()

,id,target
0,1000000,0.728627
1,1000001,0.231918
2,1000002,0.907185
3,1000003,0.836292
4,1000004,0.262122
